In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm
import time
import os

# ===== ローカル LLM の設定 =====
LLM_URL = "http://127.0.0.1:1234/v1/chat/completions"
LLM_MODEL = "llama-3.1-8b-instruct"

# ===== CSV を読み込み =====
csv_path = "コマンド推論用_1000.csv"   # ← ノートブックと同じフォルダにある前提
df = pd.read_csv(csv_path, encoding="utf-8")

# ===== 要約関数 =====
def summarize_zenbun(text: str, idx: int) -> str:
    if not isinstance(text, str) or text.strip() == "":
        print(f"[{idx}] 空行 → スキップ")
        return ""

    system_prompt = (
        "あなたは日本語の要約アシスタントです。"
        "入力された文章から、固有名詞・専門用語・数値など重要な単語を残して要約してください。"
        "出力は文章のみ。"
    )

    user_prompt = f"次の文章を要約してください。\n\n{text}"

    payload = {
        "model": LLM_MODEL,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        "temperature": 0.2,
        "max_tokens": 256,
    }

    start = time.time()
    try:
        res = requests.post(LLM_URL, json=payload, timeout=60)
        res.raise_for_status()
        data = res.json()
        summary = data["choices"][0]["message"]["content"].strip()
        elapsed = time.time() - start

        print(f"[{idx}] 完了 ({elapsed:.2f} sec): {summary[:30]}...")

        return summary

    except Exception as e:
        elapsed = time.time() - start
        print(f"[{idx}] ⚠️ LLM error ({elapsed:.2f} sec): {e}")
        return ""

# ===== zenbun 列を1行ずつ処理 =====
summaries = []
total_start = time.time()

for idx, text in tqdm(enumerate(df["zenbun"], start=1), total=len(df)):
    summary = summarize_zenbun(text, idx)
    summaries.append(summary)

total_elapsed = time.time() - total_start
avg_per_row = total_elapsed / max(len(df), 1)

df["zenbun_summary"] = summaries

# ===== 保存 =====
out_path = "コマンド推論用_1000_with_summary.csv"  # ← ローカルLLMフォルダ直下に保存
df.to_csv(out_path, index=False, encoding="utf-8")

print("\n---- 全体処理完了 ----")
print(f"総処理時間: {total_elapsed:.2f} sec")
print(f"1行あたり平均: {avg_per_row:.2f} sec/row")
print("保存先:", os.path.abspath(out_path))


  0%|          | 0/1000 [00:00<?, ?it/s]

[1] 完了 (25.15 sec): 調査目的と調査概要
調査目的：ベント管下部周辺からの漏えい状...
[2] 完了 (10.02 sec): 原子力発電所の新規制基準適合性に関する審査会議が開催されまし...
[3] 完了 (13.49 sec): 原子力規制委員会の第２回審査会合は、次の議題について議論され...
[4] 完了 (10.41 sec): 原子力規制委員会の第3回審査会合は、以下の議題について議論が...
[5] 完了 (12.14 sec): 原子力規制委員会の第4回審査会合は、次の議題について議論され...
[6] 完了 (14.78 sec): 原子力規制委員会の第５回審査会合は、次の内容で行われました。...
[7] 完了 (16.15 sec): 原子力規制委員会の第６回審査会合で議題となった内容は以下の通...
[8] 完了 (11.21 sec): 原子力発電所の新規制基準適合性に関する第7回審査会合が開催さ...
[9] 完了 (4.24 sec): 原子力規制委員会の第８回審査会合が開催されました。議題は基準...
[10] 完了 (13.48 sec): 原子力規制委員会の第9回審査会合は、次のような議題が討論され...
[11] 完了 (11.06 sec): 原子力規制委員会の第10回審査会合は、次の議題で行われた。
...
[12] 完了 (10.14 sec): 原子力規制委員会の第１１回審査会合は、次の内容で行われた。
...
[13] 完了 (16.26 sec): 原子力規制委員会の第１２回審査会合で、以下の議題が討論された...
[14] 完了 (12.23 sec): 原子力規制委員会の第13回審査会合は、以下の議題について議論...
[15] 完了 (14.05 sec): 原子力発電所の新規制基準適合性についての審査会合が14回目で...
[16] 完了 (15.18 sec): 原子力規制委員会の第15回審査会合は、次の議題を扱った。

...
[17] 完了 (10.84 sec): 原子力規制委員会の第16回審査会合は、以下の議題を扱った。
...
[18] 完了 (14.94 sec): 原子力規制委員会の第１７回審査会合は、次の議題で行われた。
...
[19] 完了 (13.66 sec):